# Classification of Mushrooms using Machine Learning | Machine learning 

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import pickle
import datetime

### Running on GPUs

In [3]:


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11529907392229525487
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6957301760
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14318422292747744572
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:26:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
Default GPU Device: /device:GPU:0
1 Physical GPUs, 1 Logical GPU


In [4]:
X  = pickle.load(open("./X.pickle","rb"))
y  = pickle.load(open("./y.pickle","rb"))
y = np.array(y)
X = X/255.0
conv_size = 16
number_of_classes =len(set(y))
#print(number_of_classes)
#print(X,y)
NAME = f"Mushrooms-cnn-256x128x64x{number_of_classes}-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
print(NAME)
tensorboard = TensorBoard(log_dir = f"logs/{NAME}")


Mushrooms-cnn-256x128x64x9-20230403-233037


### Model

In [5]:
%pip install -U scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


#Converty true labels y to one-hot econded vectors
y = to_categorical(y,num_classes = number_of_classes)
X = X.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

model = Sequential(
    [
        Conv2D(256,kernel_size = (3,3),activation = 'relu', input_shape=(X.shape[1],X.shape[2],X.shape[3])),
        MaxPooling2D(pool_size =(2,2)),
        Flatten(),
        Dense(128,activation = 'relu'),
        Dense(64,activation = 'relu'),
        Dense(32,activation = 'relu'),
        #Dense(16,activation = 'relu'),

        Dense(number_of_classes,activation='softmax')
    ]
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 256)     2560      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 256)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2509056)           0         
                                                                 
 dense (Dense)               (None, 128)               321159296 
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                        

In [7]:
#model.fit(X, y, epochs=5, validation_split=0.1,callbacks=[tensorboard],use_multiprocessing=True)
model.fit(X_train,y_train,epochs=10,batch_size=2,validation_split=0.2,callbacks=[tensorboard],use_multiprocessing=True)

Epoch 1/10
1799/1799 [==============================] - 198s 109ms/step - loss: 2.1476 - accuracy: 0.2518 - val_loss: 2.1189 - val_accuracy: 0.1544
Epoch 2/10
1799/1799 [==============================] - 188s 104ms/step - loss: 1.6509 - accuracy: 0.4180 - val_loss: 2.0653 - val_accuracy: 0.2867
Epoch 3/10
1799/1799 [==============================] - 183s 102ms/step - loss: 0.5290 - accuracy: 0.8266 - val_loss: 3.8059 - val_accuracy: 0.2889
Epoch 4/10
1799/1799 [==============================] - 190s 106ms/step - loss: 0.1438 - accuracy: 0.9600 - val_loss: 4.4796 - val_accuracy: 0.2344
Epoch 5/10
1799/1799 [==============================] - 194s 108ms/step - loss: 0.1116 - accuracy: 0.9789 - val_loss: 4.5067 - val_accuracy: 0.2344
Epoch 6/10
1799/1799 [==============================] - 188s 105ms/step - loss: 0.0758 - accuracy: 0.9872 - val_loss: 4.2965 - val_accuracy: 0.2300
Epoch 7/10
1799/1799 [==============================] - 188s 104ms/step - loss: 0.0410 - accuracy: 0.9939 - val_

In [8]:
model.evaluate(X_test,y_test, verbose = 1)

70/70 [==============================] - 3s 37ms/step - loss: 10.5095 - accuracy: 0.2076


[10.509452819824219, 0.20758122205734253]